## Web Scraping from data source

In [ ]:
# dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [ ]:
# report generator url
url = "http://www9.health.gov.au/cda/source/rpt_2_sel.cfm"

In [ ]:
# use splinter to navigate to page
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [ ]:
# set query ranges and empty lists for data return
queryyears = [2015,2016,2017,2018,2019,2020]
diseaselist =[]
diseasedflist=[]
diseasegroupings = []
eachgrouplist = []
summarytable = pd.DataFrame(columns = ["Disease_Name","Infection_Rate","Location","Disease_Group","Year"])

In [ ]:
# loop through years for field options
for year in queryyears:
    
    # use splinter to select and submit form based on field options
    browser.find_by_id('report_option_5').first.click()
    browser.find_by_id('sel_year').first.select(str(year))
    browser.find_by_id('CTIME1').click() 
    
    #get html table data into dataframe using pandas
    tables = pd.read_html(browser.html)
    df = tables[0]
    dropped = df.dropna(axis=0)
    
    #form initial dataframes on first year
    if year == 2015:
        #get list of disease groups
        diseasegroups = dropped.sort_values(by=['ACT'],ascending=False)[:8]
        diseasegroups = diseasegroups["ACT"].sort_index()
        indexlist = diseasegroups.index
        diseaselist = diseasegroups.tolist()
        diseasedflist = diseasegroups.tolist()        

        # loop through dataframe and slice based on disease groupings
        for i in range(len(indexlist)):
            if i !=7:
                dlist = dropped[indexlist[i]:indexlist[i+1]-1]["Unnamed: 0"]
            else:
                dlist = dropped[indexlist[i]:]["Unnamed: 0"]
            
            # get group and diseases paired for later database load
            dlist = dlist.values.tolist()
            for d in dlist:
                diseasegroupings.append(diseaselist[i])
                eachgrouplist.append(d)

            
        
        #slice and append data to dataframe per disease grouping
        for i in range(len(diseaselist)):
            if i !=7:
                item = dropped[indexlist[i]:indexlist[i+1]-1]
            else:
                item = dropped[indexlist[i]:]
            
            locationlist = ["ACT","NSW","NT","QLD","SA","TAS","VIC","WA","Aust","Last 5yearsmean"]
            for location in locationlist:
                state = item[["Unnamed: 0",location]]
                state["Location"] = [location for x in range(len(state))]
                state["Disease_Group"] = [diseaselist[i] for x in range(len(state))]
                state["Year"] = [year for x in range(len(state))]
                state.rename(columns = {'Unnamed: 0':"Disease_Name", location:"Infection_Rate"}, inplace = True)
                summarytable = summarytable.append(state, ignore_index=True)
               

    #loop through rest of years to append to dataframes
    else:
        
        
        for i in range(len(diseaselist)):
            if i !=7:
                item = dropped[indexlist[i]:indexlist[i+1]-1]
            else:
                item = dropped[indexlist[i]:]
                
            locationlist = ["ACT","NSW","NT","QLD","SA","TAS","VIC","WA","Aust","Last 5yearsmean"]
            for location in locationlist:
                state = item[["Unnamed: 0",location]]
                state["Location"] = [location for x in range(len(state))]
                state["Disease_Group"] = [diseaselist[i] for x in range(len(state))]
                state["Year"] = [year for x in range(len(state))]
                state.rename(columns = {'Unnamed: 0':"Disease_Name", location:"Infection_Rate"}, inplace = True)
                summarytable = summarytable.append(state, ignore_index=True)

    
    # close year report so loop can run another report
    browser.find_by_id('close').click() 

        
        

In [ ]:
summarytable

In [ ]:
browser.quit()

In [ ]:
## create sqlite database

from sqlalchemy import create_engine

path = f"sqlite:///diseases.sqlite"
engine = create_engine(path)

connection = engine.connect()



In [ ]:
#add table to database
summarytable.to_sql("DiseaseSummary",connection, if_exists='replace',index=True)

In [ ]:
## add primary key 

connection.execute("PRAGMA foreign_keys=off;")

connection.execute("BEGIN TRANSACTION;")

connection.execute("ALTER TABLE DiseaseSummary RENAME TO old_table;")

connection.execute("CREATE TABLE DiseaseSummary (id INTEGER NOT NULL PRIMARY KEY, 'Disease_Name' TEXT NOT NULL, 'Infection_Rate' REAL NOT NULL, 'Location' TEXT NOT NULL, 'Disease_Group' TEXT NOT NULL, Year INTEGER NOT NULL);")

connection.execute("INSERT INTO DiseaseSummary SELECT * FROM old_table;")

connection.execute("DROP TABLE old_table;")

connection.execute("PRAGMA foreign_keys=on;")

In [ ]:
# connection.execute("DROP TABLE Bloodbornediseases;")